# Stocks Time Series Analysis and Forecasting

In [ ]:
import os
os.chdir('D:/source/repos')
os.getcwd()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import datetime as dt
import seaborn as sb; sb.set()
import yahoo_fin.stock_info as si
import plotly.graph_objects as go
import plotly.express as px
import statsmodels.tsa.statespace.sarimax as sms
from trading.utils.StockFcst import *
from trading.utils.TsStats import *

%run ds_topics/28_time_series_forecasting/basic_methods.ipynb
%run ProblemSolving/Visual/TimeSeries/TsPlots.py

### Load data

In [ ]:
df_sp500_180 = pd.read_csv('D:/Invest/data/stock_hist/')
#df_sp500_180

sp500_candidates = pd.read_excel(('D:/Invest/data/stock_hist/'))
#sp500_candidates['ticker'].values

dict_candidates = { i : 0 for i in sp500_candidates['ticker'].values }
#dict_candidates

### MA Crosses

In [ ]:
PlotStockMA(df_sp500_180, 'TFC', 6, 30)

In [ ]:
for i in range(1, df_sp500_180.shape[1]):
    ticker = df_sp500_180.columns[i]
    if ticker in dict_candidates:
        PlotStockMA(df_sp500_180, ticker, 6, 30)

### Winners/Loosers in a period

In [ ]:
nTop = 10

res = pd.DataFrame(columns=['ticker', 'diff'])
for i in range(1, df_sp500_180.shape[1]):
    ticker = df_sp500_180.columns[i]
    try:
        diff = GetMADiff(df_sp500_180, ticker, 6, 4)
        res = res.append({ 'ticker' : ticker, 'diff': diff}, ignore_index=True)
    except:
        print('Error : ', ticker)
res.sort_values(by=['diff'], ascending=False,  inplace=True)
res

#### Winners

In [ ]:
f, ax = plt.subplots(figsize=(6, 15))
sb.barplot(x="diff", y="ticker", data=res.head(nTop))
sb.despine(left=True, bottom=True)

#### Loosers

In [ ]:
f, ax = plt.subplots(figsize=(6, 15))
sb.barplot(x="diff", y="ticker", data=res.tail(nTop))
sb.despine(left=True, bottom=True)

In [ ]:
nTop = 10
start = '2020-01-20'
end = '2020-06-26'
res = pd.DataFrame(columns=['ticker', 'maxLoss', 'currLoss'])
for i in range(1, df_sp500_180.shape[1]):
    ticker = df_sp500_180.columns[i]
    try:
        ts = df_sp500_180[ticker] 
        maxLoss = GetMaxLoss(ts)
        currLoss = GetCurrLoss(ts)
        res = res.append({ 'ticker' : ticker, 'maxLoss': maxLoss, 'currLoss': currLoss}, ignore_index=True)
    except:
        print('Error : ', ticker)
res.sort_values(by=['maxLoss'], ascending=False,  inplace=True)
res

In [ ]:
ts = df_sp500_180['AMZN']
maxLoss = GetMaxLoss(ts)
currLoss = GetCurrLoss(ts)
currLoss